#### How to choose the granularity of the context such that the agent can choose spans

In [1]:
import nltk
import spacy
import benepar

from datasets import load_dataset

squad = load_dataset("squad_v2")

nlp = spacy.load("en_core_web_trf")
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

Reusing dataset squad_v2 (/Users/teo/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
squad_sample = squad["train"][0]["context"]

In [3]:
doc = nlp(squad_sample)

# Test the first sentence
curr_sent = None
for sent in doc.sents:
    curr_sent = sent
    break

# # # create nltk tree from constituency parse
# tree = nltk.Tree.fromstring(curr_sent._.parse_string)
# tree

# # level in the constituency tree
# level = 0
# print("Granularity level: {}".format(level))
# print("Sentence: {}".format(curr_sent))

# # level order traversal of the constituency tree
# for const in curr_sent._.constituents:
#     print(const)


/opt/homebrew/anaconda3/envs/m1_ml/lib/python3.10/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/opt/homebrew/anaconda3/envs/m1_ml/lib/python3.10/site-packages/torch/distributions/distribution.py:44: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(f'{self.__class__} does not define `arg_constraints`. ' +


In [4]:
print("Level 0")
print("Sentence: {}".format(curr_sent))

print("Level 1")
result = ''
for chld in curr_sent._.children:
    result += chld.text + '#'
print("Sentence: {}".format(result))

print("Level 2")
result = ''
for chld in curr_sent._.children:
    chld2_list = []
    for chld2 in chld._.children:
        chld2_list.append(chld2.text)

    if len(chld2_list) > 0:
        result += '#'.join(chld2_list) + '#'
    else:
        result += chld.text + '#'
print("Sentence: {}".format(result))

print("Level 3")
result = ''
for chld in curr_sent._.children:
    chld2_list = []
    for chld2 in chld._.children:
        chld3_list = []
        for chld3 in chld2._.children:
            chld3_list.append(chld3.text)

        if len(chld3_list) > 0:
            chld2_list.append('#'.join(chld3_list))
        else:
            chld2_list.append(chld2.text)

    if len(chld2_list) > 0:
        result += '#'.join(chld2_list) + '#'
    else:
        result += chld.text + '#'
print("Sentence: {}".format(result))

print("Level 4")
result = ''
for chld in curr_sent._.children:
    chld2_list = []
    for chld2 in chld._.children:
        chld3_list = []
        for chld3 in chld2._.children:
            chld4_list = []
            for chld4 in chld3._.children:
                chld4_list.append(chld4.text)

            if len(chld4_list) > 0:
                chld3_list.append('#'.join(chld4_list))
            else:
                chld3_list.append(chld3.text)

        if len(chld3_list) > 0:
            chld2_list.append('#'.join(chld3_list))
        else:
            chld2_list.append(chld2.text)

    if len(chld2_list) > 0:
        result += '#'.join(chld2_list) + '#'
    else:
        result += chld.text + '#'

print("Sentence: {}".format(result))


Level 0
Sentence: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress.
Level 1
Sentence: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981)#is an American singer, songwriter, record producer and actress#.#
Level 2
Sentence: Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say)#(#born September 4, 1981#)#is#an American singer, songwriter, record producer and actress#.#
Level 3
Sentence: Beyoncé Giselle Knowles-Carter#(#/biːˈjɒnseɪ/ bee-YON-say#)#(#born#September 4, 1981#)#is#an#American#singer, songwriter, record producer and actress#.#
Level 4
Sentence: Beyoncé#Giselle#Knowles#-#Carter#(#/biːˈjɒnseɪ/#bee-YON-say#)#(#born#September#4#,#1981#)#is#an#American#singer#,#songwriter#,#record#producer#and#actress#.#


In [14]:
from collections import deque

def get_level_order_traversal(tree):
    result = []
    queue = deque()
    queue.append(tree)
    while len(queue) > 0:
        curr_node = queue.popleft()
        result.append(curr_node)
        result.append('#')
        chld_list = []
        
        for chld in curr_node._.children:
            chld_list.append(chld)

        if len(chld_list) == 0:
            chld_list = [curr_node]

        queue.extend(chld_list)



    return result

get_level_order_traversal(curr_sent)

KeyboardInterrupt: 